<center>
<img src="https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/afrisenti-logo.png" width="30%" />
</center>

In [1]:
import argparse
import os

# parser = argparse.ArgumentParser()
# parser.add_argument("--lang_code",
#                         default='am',
#                         type=str,
#                         help="Valid codes: 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'")
# parser.add_argument("--use_en", 
#                         action="store_true",
#                         help="Enable to use english data for zero shot rather than the original language codes")

# args = parser.parse_args()

# LANGUAGE_CODE = args.lang_code
# USE_EN = args.use_en

LANGUAGE_CODE = 'am'
USE_EN = True

In [2]:
print("Language Code: ", LANGUAGE_CODE)

Language Code:  am


In [3]:
import pandas as pd
import numpy as np

# Please don not edit anything here
languages = ['am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo']

colab = False


TASK = 'SubtaskA'

In [4]:
# import os

# if colab:
#     from google.colab import drive
#     drive.mount('/content/drive')
#     proj_folder = '/content/drive/MyDrive'
# else:
#     proj_folder = os.getcwd()

# %cd {proj_folder}


# PROJECT_DIR = f'{proj_folder}/afrisent-semeval-2023'
# if not os.path.isdir(PROJECT_DIR):
#   %run Make_Datasets.py

In [5]:
# MODEL_NAME_OR_PATH = 'Davlan/afro-xlmr-mini'
MODEL_NAME_OR_PATH = 'xlm-roberta-base'
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = 2
LEARNING_RATE = 5e-5
NUMBER_OF_TRAINING_EPOCHS = 5
MAXIMUM_SEQUENCE_LENGTH = 128
SAVE_STEPS = -1



####Starter Code: Datasets, etc


In [6]:
from types import SimpleNamespace
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import warnings
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, balanced_accuracy_score
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
import pandas
import pandas as pd
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    PreTrainedTokenizerFast,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from tokenizers import SentencePieceBPETokenizer
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from datasets import Features, Value, ClassLabel, load_dataset, Dataset

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/text-classification/requirements.txt")

np.random.seed(69)
torch.manual_seed(420);

NOTE: Redirects are currently not supported in Windows or MacOs.


In [7]:
# folder = ''


# if colab:
#     from google.colab import drive
#     drive.mount('/content/drive')
#     proj_folder = '/content/drive/MyDrive'
# else:
#     proj_folder = os.getcwd()

# %cd {proj_folder}


# PROJECT_DIR = f'{proj_folder}/afrisent-semeval-2023'


PROJECT_DIR = 'afrisent-semeval-2023'

TRAINING_DATA_DIR = os.path.join(PROJECT_DIR, TASK, 'train')
FORMATTED_TRAIN_DATA = os.path.join(TRAINING_DATA_DIR, 'formatted-train-data')

TRAINING_DATA_DIR

'afrisent-semeval-2023\\SubtaskA\\train'

In [8]:
DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', LANGUAGE_CODE)

In [9]:
LANGUAGE_CODE

'am'

In [10]:
# Set seed before initializing model.
set_seed(42069)

# obtain dev data
df = pd.concat([pd.read_csv(DATA_DIR + '/dev.tsv', sep='\t'), pd.read_csv(DATA_DIR + '/test.tsv', sep='\t')])
df = df.dropna()
eval_dataset = Dataset.from_pandas(df)
label_list = df['label'].unique().tolist()

# Labels
num_labels = len(label_list)
print(label_list)

['neutral', 'positive', 'negative']


In [11]:

# obtain train data
if USE_EN:
    print('Using EN for zero shot')
    df = pd.read_csv('../adapter_notebooks/data/en_all.csv')[['text', 'labels']].rename(
        columns={'labels':'label'})
else:
    lang_trains = []
    for lang in languages:
        if lang != LANGUAGE_CODE:
            
            lang_data_dir = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', lang)
            print('Adding', lang, 'to the training set')
            lang_trains.append(pd.read_csv(lang_data_dir + '/train.tsv', sep='\t'))
    df = pd.concat(lang_trains)

df = df.dropna()
train_dataset = Dataset.from_pandas(df)

Using EN for zero shot


In [12]:
train_dataset[0]

{'text': 'Happy birthday Roman Abramovich. Treat yourself to a nice present in January. I think Radamel Falcao would fit the bill nicely.',
 'label': 'positive'}

####Tokenization

In [13]:
config = AutoConfig.from_pretrained(
    MODEL_NAME_OR_PATH,
    num_labels=num_labels,
    cache_dir=None,
    revision='main',
    use_auth_token=None,
)
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME_OR_PATH,
    do_lower_case=None,
    cache_dir=None,
    use_fast=True,
    revision='main',
    use_auth_token=None,
)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME_OR_PATH,
    from_tf=False,
    config=config,
    cache_dir=None,
    revision="main",
    use_auth_token=None,
    ignore_mismatched_sizes=False,
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [14]:
# Preprocessing the datasets
# Padding strategy
padding = "max_length"


label_to_id = None
label_to_id = {v: i for i, v in enumerate(label_list)}



In [15]:
def preprocess_function(examples):
    texts =(examples['text'],)
    result = tokenizer(*texts, padding=padding, max_length=MAXIMUM_SEQUENCE_LENGTH, truncation=True)
    print(len(result['input_ids']), len(result['input_ids'][0]))
    
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    
    result['length'], result["tokenized"] = [], []
    for input_ids in result['input_ids']:
        toks = tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
        result['length'].append(len(toks)+2)
        result['tokenized'].append(' '.join(toks))
    return result

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on train dataset",
)

eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on validation dataset",
)

Running tokenizer on train dataset:   0%|          | 0/13 [00:00<?, ?ba/s]

1000 128
1000 128
1000 128
1000 128
1000 128
1000 128
1000 128
1000 128
1000 128
1000 128
1000 128
1000 128
721 128


Running tokenizer on validation dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

1000 128
796 128


In [16]:
train_dataset.filter(lambda a: len(a['input_ids']) <= MAXIMUM_SEQUENCE_LENGTH)

  0%|          | 0/13 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask', 'length', 'tokenized'],
    num_rows: 12721
})

In [17]:
len(train_dataset)

12721

In [18]:
train_dataset[0]

{'text': 'Happy birthday Roman Abramovich. Treat yourself to a nice present in January. I think Radamel Falcao would fit the bill nicely.',
 'label': 1,
 'input_ids': [0,
  32506,
  101207,
  12610,
  168044,
  145739,
  5,
  4804,
  257,
  31949,
  47,
  10,
  26267,
  13379,
  23,
  18982,
  5,
  87,
  5351,
  95041,
  7603,
  22225,
  123142,
  2806,
  11177,
  70,
  54727,
  26267,
  538,
  5,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [19]:
eval_dataset[0]

{'text': '?? ?? ወርቅ እንደሚዘንብ ጠብቀህ ነበር ?? ለዛውም በስዕታት ውስጥ አየህ የዚህን ያህል ነው ሰውየውን የምትፈሩት....እኔ እኮ የማዝነው አንተን ፋለው ለሚያደርጉ ሰወች ነው ምስኪን ፂም!',
 'label': 0,
 '__index_level_0__': 0,
 'input_ids': [0,
  24610,
  24610,
  6,
  122109,
  16678,
  117225,
  3833,
  6,
  91284,
  4656,
  4359,
  25880,
  24610,
  2237,
  13075,
  63083,
  48344,
  16160,
  53261,
  5405,
  2350,
  5186,
  4359,
  144382,
  548,
  55620,
  3053,
  13698,
  46506,
  548,
  59581,
  6980,
  63618,
  5,
  27,
  81970,
  158171,
  22204,
  7872,
  26190,
  97081,
  548,
  6,
  8036,
  13818,
  2237,
  52534,
  92691,
  12560,
  5698,
  5519,
  3053,
  61075,
  29654,
  548,
  6,
  244044,
  816,
  38,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  

In [20]:
train_dataset, eval_dataset

(Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask', 'length', 'tokenized'],
     num_rows: 12721
 }),
 Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask', 'length', 'tokenized'],
     num_rows: 1796
 }))

In [21]:
train_text, train_labels = train_dataset['tokenized'], train_dataset['label']

In [22]:
eval_text, eval_labels = eval_dataset['tokenized'], eval_dataset['label']

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [26]:
# Get the metric function
metric = evaluate.load("accuracy")

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)


data_collator = default_data_collator

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args = TrainingArguments(
        do_train=True,
        do_eval=True,
        per_device_train_batch_size=BATCH_SIZE,
        learning_rate=LEARNING_RATE,
        num_train_epochs=NUMBER_OF_TRAINING_EPOCHS,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        output_dir='tmp_trainer',
        save_steps=SAVE_STEPS,
        overwrite_output_dir=True
    ),
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)
# Training

train_result = trainer.train(resume_from_checkpoint=None)
metrics = train_result.metrics
metrics["train_samples"] = len(train_dataset) 


# Evaluation

metrics = trainer.evaluate(eval_dataset=eval_dataset)

metrics["eval_samples"] = len(eval_dataset)

for key,value in metrics.items():
    print(key, ':', value)

splitted_A = os.path.join(PROJECT_DIR, 'SubtaskA', 'train', 'splitted-train-dev-test')

try:
    LANGUAGE_CODE
except NameError:
    LANGUAGE_CODE = 'combined'
else:
    pass

data = []
for lang in languages:
    eval_path = os.path.join(splitted_A, lang)
    df = pd.read_csv(eval_path + '/dev.tsv', sep='\t')
    df = df.dropna()
    lang_eval = Dataset.from_pandas(df)
    lang_eval = lang_eval.map(
        preprocess_function,
        batched=True,
        load_from_cache_file=True,
        desc="Running tokenizer on validation dataset",
    )

    predictions, labels, metrics = trainer.predict(lang_eval, metric_key_prefix="eval")

    if LANGUAGE_CODE == lang:
        f1 = (f1_score(labels, np.argmax(predictions, axis=1), average='weighted'))
        bal_acc = balanced_accuracy_score( labels, np.argmax(predictions, axis=1))

    data.append([LANGUAGE_CODE, lang, str(list(predictions)), str(list(labels))])
df = pd.DataFrame(data, columns=['source', 'target', 'predictions', 'labels'])
df.to_csv(f'{LANGUAGE_CODE}_preds.csv', index=False)


trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)
print(f"f1 score: {f1:.3}, balanced acc: {bal_acc:.3}")

print(df)

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text, tokenized, length. If text, tokenized, length are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Thomas\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12721
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1990


Step,Training Loss



KeyboardInterrupt



In [ ]:
# import os
# %cd {PROJECT_DIR}

In [ ]:
# DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', LANGUAGE_CODE)
# OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE + '_no_test')
# kinya = 'jean-paul/KinyaBERT-small'

# !python starter_kit/run_textclass.py \
#   --model_name_or_path {kinya} \
#   --data_dir {DATA_DIR} \
#   --do_train \
#   --do_eval \
#   --per_device_train_batch_size {BATCH_SIZE} \
#   --learning_rate {MAXIMUM_SEQUENCE_LENGTH} \
#   --num_train_epochs {NUMBER_OF_TRAINING_EPOCHS} \
#   --max_seq_length {MAXIMUM_SEQUENCE_LENGTH} \
#   --output_dir {'tmp_trainer'} \
#   --save_steps {SAVE_STEPS} \
#   --overwrite_output_dir